## Deliverable 2. Create a Customer Travel Destinations Map.

In [52]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [54]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,73.99,39,0,6.93,clear sky
1,1,Yellowknife,CA,62.46,-114.35,-2.99,76,20,10.29,few clouds
2,2,Otjiwarongo,NaN,-20.46,16.65,73.40,55,53,2.24,broken clouds
3,3,Taiyuan,CN,37.87,112.56,28.40,68,100,2.24,haze
4,4,Ushuaia,AR,-54.80,-68.30,39.20,69,40,5.82,scattered clouds


In [55]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [56]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,73.99,39,0,6.93,clear sky
2,2,Otjiwarongo,NaN,-20.46,16.65,73.40,55,53,2.24,broken clouds
10,10,Paraiso,CR,9.84,-83.87,71.60,88,75,3.36,broken clouds
11,11,Bathsheba,BB,13.22,-59.52,77.00,88,40,14.99,scattered clouds
13,13,New Norfolk,AU,-42.78,147.06,73.99,35,85,7.00,overcast clouds
14,14,Saint-Philippe,RE,-21.36,55.77,71.60,68,0,5.82,clear sky
22,22,Avarua,CK,-21.21,-159.78,80.60,69,44,18.34,scattered clouds
25,25,Vung Tau,VN,10.35,107.08,80.06,72,100,2.91,overcast clouds
26,26,Tabuk,PH,17.42,121.44,73.87,92,100,8.63,moderate rain
31,31,Rikitea,PF,-23.12,-134.97,75.83,80,69,17.90,broken clouds


In [57]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                237
City                   237
Country                236
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [58]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = cities_df.dropna()
clean_cities_df.count()

City_ID                163
City                   163
Country                163
Lat                    163
Lng                    163
Max Temp               163
Humidity               163
Cloudiness             163
Wind Speed             163
Current Description    163
dtype: int64

In [59]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Bathsheba,BB,77.00,scattered clouds,13.22,-59.52,NaN
22,Avarua,CK,80.60,scattered clouds,-21.21,-159.78,NaN
25,Vung Tau,VN,80.06,overcast clouds,10.35,107.08,NaN
31,Rikitea,PF,75.83,broken clouds,-23.12,-134.97,NaN
37,Kralendijk,BQ,80.60,overcast clouds,12.15,-68.27,NaN
39,Faanui,PF,81.25,broken clouds,-16.48,-151.75,NaN
43,Yenagoa,NG,75.18,clear sky,4.92,6.26,NaN
44,San Carlos Del Zulia,VE,75.56,light rain,9.00,-71.91,NaN
45,Inhambane,MZ,76.10,light rain,-23.86,35.38,NaN
51,Vaini,TO,84.20,scattered clouds,-21.20,-175.20,NaN


In [60]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [61]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   154
Country                154
Max Temp               154
Current Description    154
Lat                    154
Lng                    154
Hotel Name             154
dtype: int64

In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))